In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-12-16


Found rundate 2023-12-15 10:00:02 < 2023-12-16 at commit 790d688d; returning commit a3176fd


'2023-12-16 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-12-16 10:00:03, NJSP was reporting 577 YTD deaths
Current YTD Deaths (2024-12-16 10:00:00-05:00): 667
Previous year YTD Deaths (adjusted; 2023-12-16 10:00:03): 577.070251744572
Projected 2024 total: 700.246161475884
15.6% change


FAUQStats cache miss: 2023, Sat Dec 16 10:00:03 EST 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,0.0,Mulberry St,2023-01-01 01:15:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Road,1.0,1.0,0.0,0.0,0.0,0.0,South Randolphville Road,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,0.0,Outwater Ln,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,Washington St,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12869,12,Middlesex,1213,Monroe Twsp,522,County 522 MP 13.05,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-12-09 14:09:00-05:00
12870,20,Union,2004,Elizabeth City,1,State Highway 1,2.0,1.0,1.0,0.0,0.0,NaN,NaN,2023-12-10 05:49:00-05:00
12871,06,Cumberland,0605,Fairfield Twsp,670,County 670 MP 2.1,1.0,0.0,1.0,0.0,0.0,3.0,NaN,2023-12-10 21:55:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,29,2,18,4,52
Bergen,15,5,19,2,39
Burlington,31,4,8,1,44
Camden,18,3,15,0,36
Cape May,8,0,0,0,8
Cumberland,19,6,6,0,26
Essex,17,10,27,0,48
Gloucester,17,8,6,0,28
Hudson,9,2,5,1,16


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

95.7% through the year, 4.3% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,16,18,29
Bergen,driver,22,20,15
Burlington,driver,24,26,31
Camden,driver,18,18,18
Cape May,driver,4,4,8
...,...,...,...,...
Salem,crashes,10,11,13
Somerset,crashes,20,21,18
Sussex,crashes,6,6,13


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,57,8,33,2,18
Bergen,38,2,14,5,21
Burlington,50,2,34,6,8
Camden,39,0,18,4,18
Cape May,9,0,8,0,0
Cumberland,29,0,21,6,8
Essex,53,0,19,16,28
Gloucester,31,0,18,11,7
Hudson,16,2,8,2,5


In [9]:
projected.to_csv(PROJECTED_CSV)